In [29]:
# uncomment on first run to install qiskit library
##############################################
# !pip install openqasm3
# !pip install pip install qiskit-qasm3-import
# !pip install qiskit
# !pip install qiskit-ibm-runtime
##############################################

#initialization
import matplotlib.pyplot as plt  # plotting the results
import numpy as np               # create and manage arrays
# import math                      # using more complexe math operations

# importing Qiskit
from qiskit import IBMQ, Aer, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.transpiler import CouplingMap
import qiskit.quantum_info as qi
from qiskit.tools.monitor import job_monitor

# import for the qasm parsing
from qiskit import qasm3
from qiskit_qasm3_import import parse

# import basic plot tools
from qiskit.visualization import plot_histogram, array_to_latex, plot_gate_map
from qiskit.circuit.library import MCMT, MCMTVChain, Diagonal

# import math tools
from math import floor, ceil, sqrt, pi, log2, cos, sin   # using more complexe math operations

In [10]:
provider = IBMQ.load_account()

provider.backends()  # list of backends

backend = provider.get_backend("ibmq_belem")
cm = CouplingMap(backend.configuration().coupling_map) # get the coupling map of the processor
plot_gate_map(backend)  # plot the layout of the processor

print(backend.configuration().basis_gates)

ibmqfactory.load_account:WARNING:2023-07-24 12:00:40,501: Credentials are already in use. The existing account in the session will be replaced.


['id', 'rz', 'sx', 'x', 'cx', 'reset']


In [11]:
# export qiskit to qasm3 or run

# exporter = qasm3.Exporter(
#     includes=[], disable_constants=True, basis_gates=backend.configuration().basis_gates
# )

# print(qasm3_bell := exporter.dumps(qiskit_transpiled_circuit))

# qasm3_bell_job = run_qasm3_program(qasm3_bell)

In [12]:
# import qasm3 to qiskit backend

# my_first_qasm3_program = """
# OPENQASM 3.0;

# qubit $0;
# bit a0;

# x $0;
# a0 = measure $0;

# """
# def run_qasm3_program(program_str, runtime_backend=backend):
#     shots = 1000
#     job = backend.run(program_str, shots=shots, dynamic=True)
#     print(f"Runtime job id: {job.job_id()}")
#     job_monitor(job)
#     return job
# job = run_qasm3_program(my_first_qasm3_program)

In [13]:
def ReflectionGateMatrix(theta):
    isReflect_op = qi.Operator([[cos(theta), sin(theta)], 
                                [sin(theta), - cos(theta)]])
    return isReflect_op

def run_qasm3_program(program_str, runtime_backend=backend):
    shots = 1000
    job = backend.run(program_str, shots=shots, dynamic=True)
    print(f"Runtime job id: {job.job_id()}")
    job_monitor(job)
    return job

def AddUvGate(qc, theta, q):
    Uv = ReflectionGateMatrix(theta)
    qc.unitary(Uv, q, label = 'U$_v$')

def AND(qc, r, x, y):

# Alice = qubit_0 , Bob = qubit_1

    theta = pi/(8 * r)
    iter = 4 * r - 1
    print("theta = ", theta)
    print("iter = ", iter)


    for i in range(iter - 1):
# - Alice: identity if x = 0, U if x = 1, then send to Bob
        if x == 1:
            #apply U
            AddUvGate(qc, theta, 0)
        
        #swap with Bob
        qc.swap(0, 1)
    
# - Bob: identity if y = 0, Z if y = 1, then send to Alice
        if y == 1:
            qc.z(1)
            
        #swap with Bob
        qc.swap(0, 1)    
        
# - Alice: identity if x = 0, U if x = 1, then send to Bob
    if x == 1:
        #apply U
        AddUvGate(qc, theta, 0)

    #swap with Bob
    qc.swap(0, 1)

# - Bob: identity if y = 0, Z if y = 1, then send to Alice
    if y == 1:
        qc.z(1)  
                 
        
# - Repeat Process 4r - 1 rounds, measure, if result = 1, answer = 1 if not 0
    qc.measure(1, 0)
    
 

In [14]:
r = 2
qc = QuantumCircuit(2, 1)
q0 = qc.qubits[0]
q1 = qc.qubits[1]

AND(qc, r, 1, 1)
# qc.draw('mpl')

theta =  0.19634954084936207
iter =  7


In [28]:

# exporter = qasm3.Exporter(
#     includes=[], disable_constants=True, basis_gates=backend.configuration().basis_gates
# )

qasm3_bell = qasm3.dumps(qc)

qasmTest = open("qasmTest.txt", "w")
qasmTest.write(qasm3_bell)
qasmTest.close()

# print(qasm3_bell)


TypeError: only length-1 arrays can be converted to Python scalars

In [24]:
# with open('qasmTest.txt', 'r') as file:
#     qasm = file.read()
# print(qasm)

qasm = """

OPENQASM 3;
gate u3_140478502215280(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140479034959008(_gate_p_0) _gate_q_0 {
  u3_140478502215280(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate swap _gate_q_0, _gate_q_1 {
  cx _gate_q_0, _gate_q_1;
  cx _gate_q_1, _gate_q_0;
  cx _gate_q_0, _gate_q_1;
}
gate u3_140478502222576(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0, 0, 3.141592653589793) _gate_q_0;
}
gate u1_140478502217056(_gate_p_0) _gate_q_0 {
  u3_140478502222576(0, 0, 3.141592653589793) _gate_q_0;
}
gate z _gate_q_0 {
  u1_140478502217056(3.141592653589793) _gate_q_0;
}
gate u3_140478502223824(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140478504499184(_gate_p_0) _gate_q_0 {
  u3_140478502223824(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate u3_140478502216816(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140479438420336(_gate_p_0) _gate_q_0 {
  u3_140478502216816(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate u3_140478496698960(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140478502357792(_gate_p_0) _gate_q_0 {
  u3_140478496698960(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate u3_140478476191680(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140478502356832(_gate_p_0) _gate_q_0 {
  u3_140478476191680(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate u3_140478476192016(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140479034958576(_gate_p_0) _gate_q_0 {
  u3_140478476192016(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate u3_140478476192832(_gate_p_0, _gate_p_1, _gate_p_2) _gate_q_0 {
  U(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
gate unitary_140479034951712(_gate_p_0) _gate_q_0 {
  u3_140478476192832(0.39269908169872414, 0.0, 3.141592653589793) _gate_q_0;
}
bit[1] c;
qubit[2] _all_qubits;
let q = _all_qubits[0:1];
unitary_140479034959008([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
swap q[0], q[1];
unitary_140478504499184([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
swap q[0], q[1];
unitary_140479438420336([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
swap q[0], q[1];
unitary_140478502357792([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
swap q[0], q[1];
unitary_140478502356832([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
swap q[0], q[1];
unitary_140479034958576([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
swap q[0], q[1];
unitary_140479034951712([[ 0.98078528+0.j  0.19509032+0.j]
 [ 0.19509032+0.j -0.98078528+0.j]]) q[0];
swap q[0], q[1];
z q[1];
c[0] = measure q[1];

"""

# qasm3_qc = parse(qasm)
# qasm3_qc = qasm3.loads(qasm)




line 63:24 no viable alternative at input 'unitary_140479034959008(['


QASM3ParsingError: 

In [27]:
backend = Aer.get_backend('qasm_simulator') # uncomment if you want to run on a simulator

# tr_qasm3_qc = transpile(qasm3_qc, backend)

# job = backend.run(tr_qasm3_qc)                  # run the job on the given backend
job = run_qasm3_program(qasm)
results = list(job.result().get_counts().items()) # list of all results
# threshold = int(1024/3)                           # arbitrary threshold to isolate good results
# InverseResults(results, threshold)                # inverse the position answers (qubits in qiskit are inversed)

plot_histogram(job.result().get_counts())         # uncomment to plot the result

NameError: name 'run_qasm3_program' is not defined